# ETL-pipelines in Azure Databricks
## Module 6: Automatisering en Orkestratie
In deze module leren we hoe we ETL-processen **automatisch laten draaien** in **Azure Databricks**.

## Stap 1: Start een SparkSession
We beginnen met het opzetten van een SparkSession, de kern van Apache Spark.

In [ ]:
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.appName("ETL_Automation").getOrCreate()

## Stap 2: Maak een ETL-functie
We maken een **ETL-pipeline** als een Python-functie.

In [ ]:
def run_etl():
    # Stap 1: Laad de declaraties dataset
    df = spark.read.csv("/mnt/data/declaraties_dataset.csv", header=True, inferSchema=True)
    
    # Stap 2: Opschonen (filter ongeldige bedragen)
    df_clean = df.filter((df["Bedrag"] >= 10) & (df["Bedrag"] <= 500))
    
    # Stap 3: Opslaan in Delta Lake (Zilver-laag)
    df_clean.write.format("delta").mode("overwrite").save("/mnt/data/zilver/declaraties")
    
    print("ETL Pipeline voltooid ✅")

# Run de ETL pipeline
run_etl()

## Stap 3: Logging en foutafhandeling
We loggen fouten in een Delta Lake tabel.

In [ ]:
from pyspark.sql.functions import lit
import datetime

def log_error(error_message):
    error_df = spark.createDataFrame(
        [(str(datetime.datetime.now()), error_message)], 
        ["Timestamp", "Error"]
    )
    
    error_df.write.format("delta").mode("append").save("/mnt/data/logs/errors")
    
# Test de logging
log_error("Testfout: Declaratiebestand niet gevonden")

## Stap 4: CRUD-operaties in Delta Lake
We voeren een update- en delete-operatie uit.

In [ ]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/mnt/data/zilver/declaraties")
delta_table.update(condition="Zorgtype = 'Tandarts'", set={"Bedrag": "Bedrag * 1.05"})
delta_table.delete("Bedrag < 20")

## Stap 5: Optimaliseren van Delta Lake
We verbeteren prestaties met OPTIMIZE en VACUUM.

In [ ]:
spark.sql("OPTIMIZE delta.`/mnt/data/zilver/declaraties`")
spark.sql("VACUUM delta.`/mnt/data/zilver/declaraties` RETAIN 7 HOURS")

## Oefeningen
1. **Maak een Databricks Job om deze ETL-functie dagelijks uit te voeren.**
2. **Voeg een retry-mechanisme toe aan de ETL-pipeline.**
3. **Simuleer een fout en controleer of deze correct wordt gelogd.**